In [5]:
from SPARQLWrapper import SPARQLWrapper, JSON
sparql = SPARQLWrapper("http://localhost:3030/InvestigacionesUdenar/sparql")

### Cargamos Vocabulario y Adicionales

In [3]:
import difflib

#Lectura de lemas 
import json
with open('./Vocabulario/diccionario-lemas.json', 'r') as f:
    diccionario_lemas = json.loads(f.read())

lemas = list(diccionario_lemas.keys())

### Cargamos Modelo W2V

In [4]:
from gensim.test.utils import datapath
from gensim import utils
from gensim.models import Word2Vec
import multiprocessing
from time import time

modelo = Word2Vec.load("./Modelos/modelo10FINAL.model")


### Lectura de Palabras

In [84]:
words = ['medicina', 'natural']
#Modelo W2v
similar_words = modelo.wv.most_similar(words, topn=20)

words.extend([word[0] for word in similar_words])

#Palabras en plural o muy similares
more_words=[]
for word in words:
    additional = difflib.get_close_matches(word, diccionario_lemas.keys(), n=5, cutoff=0.90)
    more_words.extend(additional[1:])
    
#Añadir palabras finales
words.extend(more_words)
words

['medicina',
 'natural',
 'netamente',
 'tautomerismo',
 'giepi',
 'cultivarse',
 'prototropico',
 'ingerir',
 'suramericanos',
 'anular',
 'setas',
 'pulso',
 'nulas',
 'mutacion',
 'ostreatus',
 'pleurotus',
 'marihuana',
 'pulverizado',
 'combatir',
 'fig',
 'carminico',
 'aprovechable',
 'cultivares']

### Cargamos palabras a SPARQL 

In [85]:
query_prefix = """
PREFIX po: <http://www.semanticweb.org/OntologiaInvestigacionPrueba#>
select ?id_pi?Titulo?proyecto_investigacion (count(?id_pi)as ?c)
where{
        ?palabras po:descripcion_palabra ?descripcion.
        ?palabras ^po:pi_tiene_palabra ?proyecto_investigacion.
        ?proyecto_investigacion po:titulo_proyecto_investigacion ?Titulo.
        ?proyecto_investigacion po:id_proyecto_investigacion ?id_pi.
        FILTER ( """

query_words=""""""
query_words_start = """REGEX(str(?descripcion), '\\\\b"""
query_words_final = """\\\\b','i') || """
query_words_final_string = """\\\\b','i')"""

for i, word in enumerate(words):
    if i < len(words)-1:
        query_words += query_words_start+word+query_words_final
    else:
        query_words += query_words_start+word+query_words_final_string
        

query_close= """)
    }
group by ?id_pi?Titulo?proyecto_investigacion
order by DESC(?c)
"""

query = query_prefix+query_words+query_close
#print(query)

### Realizamos la consulta

In [86]:
sparql.setQuery(query)

#order by DESC(?c)

sparql.setReturnFormat(JSON)

results = sparql.query().convert()
results = results['results']['bindings']

### Mostramos Resultados

In [87]:
for i, result in enumerate(results):
    print(f'({i+1}) -> ' + result['Titulo']['value'])

(1) -> DETERMINACIÓN DE PRINCIPIOS ACTIVOS MEDICINALES DEL HONGO Ganoderma lucidum CULTIVADO SOBRE RESIDUOS SÓLIDOS Y SU USO EN LA PRODUCCIÓN DE UN ALIMENTO FUNCIONAL
(2) -> EFECTO DEL SECADO POR VENTANA DE REFRACTANCIA SOBRE LA CALIDAD Y COMPUESTOS VOLÁTILES DE Pleurotus ostreatus
(3) -> ESTUDIO COMPUTACIONAL DEL EQUILIBRIO TAUTOMÉRICO EN 1,2,4-TRIAZOLES-3-AMINO-5-SUSTITUIDOS
(4) -> AGUA EXTRAVASCULAR PULMONAR COMO PREDICTOR DE MORTALIDAD Y DESARROLLO DE INJURIA PULMONAR AGUDA/SINDROME DE DISTRES RESPIRATORIO AGUDO EN PACIENTES EN ESTADO DE SHOCK . HOSPITAL UNIVERSITARIO DEPARTAMENTAL DE NARIÑO. 2013-2014
(5) -> BETALAÍNAS DEL FRUTO DE PITAYA AMARILLA (Selenicereus megalanthus): IDENTIFICACIÓN, ESTABILIDAD Y ACTIVIDAD ANTIOXIDANTE IN VITRO.
(6) -> CULTIVO DEL HONGO MEDICINAL Pleurotus ostreatus Y DETERMINACIÓN DE SU ACTIVIDAD ENZIMÁTICA Y ANTIOXIDANTE
(7) -> DESARROLLO DE PROCESOS DE EXTRACCIÓN DE CANNABINOIDES PRESENTES EN MARIHUANA (Cannabis sativa) UTILIZANDO TECNOLOGÍAS DE EXTRACC

In [88]:
id_docs = [result['id_pi']['value'] for result in results ]
len(id_docs)

57

In [2]:
qres = g.query(
    """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX po: <http://www.semanticweb.org/OntologiaInvestigacionPrueba#>
select ?id_pi?Titulo?proyecto_investigacion (count(?id_pi)as ?c)
where{
        ?proyecto_investigacion po:pi_tiene_palabra ?palabra.
        ?proyecto_investigacion po:titulo_proyecto_investigacion ?Titulo.
        ?proyecto_investigacion po:id_proyecto_investigacion ?id_pi.
        ?palabra po:descripcion_palabra ?descripcion.
        FILTER (
            REGEX(str(?descripcion), 'descubrimiento','i')||
            REGEX(str(?descripcion), 'conocimiento','i')
        )
    }
group by ?id_pi?Titulo?proyecto_investigacion
""")


In [11]:
qres = g.query(
    """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX po: <http://www.semanticweb.org/OntologiaInvestigacionPrueba#>
select ?id_pi?Titulo?proyecto_investigacion (count(?id_pi)as ?c)
where{
        ?palabras po:palabra_describe_pi ?proyecto_investigacion.
        ?proyecto_investigacion po:titulo_proyecto_investigacion ?Titulo.
        ?proyecto_investigacion po:id_proyecto_investigacion ?id_pi.
        ?palabras po:descripcion_palabra ?descripcion.
        FILTER (
            REGEX(str(?descripcion), 'descubrimiento','i')||
            REGEX(str(?descripcion), 'conocimiento','i')
        )
    }
group by ?id_pi?Titulo?proyecto_investigacion
order by DESC(?c)
""")

In [5]:
#Directa
trabajos = list(graph.query("""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX po: <http://www.semanticweb.org/OntologiaInvestigacionPrueba#>
select ?id_pi?Titulo?proyecto_investigacion (count(?id_pi)as ?c)
where{
        ?proyecto_investigacion po:pi_tiene_palabra ?palabra.
        ?proyecto_investigacion po:titulo_proyecto_investigacion ?Titulo.
        ?proyecto_investigacion po:id_proyecto_investigacion ?id_pi.
        ?palabra po:descripcion_palabra ?descripcion.
        FILTER (
            REGEX(str(?descripcion), 'descubrimiento','i')||
            REGEX(str(?descripcion), 'conocimiento','i')
        )
    }
group by ?id_pi?Titulo?proyecto_investigacion
order by DESC(?c)
"""))

#trabajos
                